In [1]:
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.0 MB/s eta 0:00:00


In [2]:
import accelerate
accelerate.__version__

'0.22.0'

In [3]:
import os
import math
import random
import pandas as pd
import regex as re
import numpy as np
from typing import Optional, Sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

from tqdm import tqdm
import torch
from torch import nn
from torch import Tensor
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, EarlyStoppingCallback, AutoModel, AutoConfig

import gc
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from google.colab import drive
drive.mount('/content/drive')##구글 코랩에 연결

Mounted at /content/drive


In [5]:
df=pd.read_csv('drive/MyDrive/FinalData.csv')
## null값 확인
df.isnull().sum()

comment    0
label      0
dtype: int64

In [6]:
#일상 대화 데이터
df2=pd.read_csv('drive/MyDrive/dialog.csv')
df2.head()

,comment,label
0,점심 메뉴 정하신 분 누룽지 끓여 먹을까 고민 나 아침 든든하게 먹으니 힘 ...,0
1,주택 사니까 좋제 키키 주택 살면 좋은 것도 있구 가드닝 책 오늘 왔당 하하...,0
2,진주에서 분밖예 안 걸리지 진주에서 평거에선 분이양 기름값이 너무 비싸서 ...,0
3,어디 취직하는 게 낫다 근데 왜 이거는 별루야 왜 낫냐 난 싫어 이거 사...,0
4,요즘에는 군대 괴롭힘 없겠지 요즘은 그러면 큰일 날 듯 직업 군인들 꿀빠는 ...,0


In [7]:
df=pd.concat([df,df2])
df.head()

,comment,label
0,"네, 흥미가 있군요. 문제가 있어서 드릴게요. 그럼 상관없어요. 계정을 이용하실 수...",1
1,"네, 안녕하세요? 네, 안녕하세요. 손님이신가요? 네, 안녕하세요. 나는 국민은행 ...",1
2,"네, 안녕하세요? 네, 직원 채용에 대해 연락주신 분입니까? 네, 먼저 저는 Bac...",1
3,네. 대출금 추정 후 연체가 없으면 원리균등분할상환 원금 이상 중 한 번 상법을 변...,1
4,"아까도 말했지만 담당자는 모른다고 했어요. 네, 잠시만요. 사회보장번호를 확인해주세...",1


In [8]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(df, test_size=0.2, random_state=2023)

print(train['label'].value_counts())
print('\n')
print(test['label'].value_counts())

0    23428
1     9850
Name: label, dtype: int64


0    5912
1    2408
Name: label, dtype: int64


In [9]:
train=train.reset_index().iloc[:,[1,2]]
test=test.reset_index().iloc[:,[1,2]]
train.head()

,comment,label
0,제 차 브레이크가 고장나 담벼락을 박았어요. 자동차보험으로 보상처리 가능한가요? ...,0
1,진짜 겨울에 눈 이쁘게 내리는 거 구경하면서 따듯하고 좋은 펜션 잡고 놀고 싶다...,0
2,네? 뭐라고요? 본인 명의들 사건 관련해 가지고 몇 가지 확인차 연락을 드렸어요 본...,1
3,니는 예능 잘 챙겨보나 저는 런닝맨 좋아합니다 오 런닝맨 키키 런닝맨은 풀로...,0
4,계절이 바껴서 그런가 몸이 간지럽네 수분 부족 아니에요 수분 부족인가 ...,0


### Preprocessing

In [10]:
#텍스트 전처리 (한글, 영어만 남기고 제거)
train['comment'] = train['comment'].apply(lambda x: re.sub("[^ A-Za-z가-힣]", "", x))
train['comment'] = train['comment'].apply(lambda x: re.sub("[ +]", " ", x))

test['comment'] = test['comment'].apply(lambda x: re.sub("[^ A-Za-z가-힣]", "", x))
test['comment'] = test['comment'].apply(lambda x: re.sub("[ +]", " ", x))

# Dataset

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'현재 device는 {device}입니다')

현재 device는 cuda입니다


In [12]:
train['comment'].str.len().max(), test['comment'].str.len().max()

(4809, 4804)

In [13]:
#device = torch.device("cuda")
model_path = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
length = 880  #800 #train['comments'].str.len().max()

### CustomDataset

In [14]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            st_bias = self.labels['is_phising'][idx]
            item["labels"] = torch.tensor(st_bias)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# HuggingFace Phase

## config

In [15]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'ej.park'
print(config.num_hidden_layers)
config

12


BigBirdConfig {
  "_name_or_path": "ej.park",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 3,
  "tokenizer_class": "BertTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.32.0",
  "type_vocab_size": 2,
  "use_bias": true,
  "use_cache": true,
  "vocab_size": 32500
}

## Custom Model

In [16]:
# define meanpooling >> pooling을 적용함으로써 노이즈에 덜 민감하게
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [17]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        if model_path == 'monologg/kobigbird-bert-base':
            config.attention_type = "original_full"
        ##model
        self.base_model = AutoModel.from_pretrained(model_path, config=config)
        self.out = self.base_model.encoder.layer[-1].output.dense.out_features

        ##pooling
        self.mean_pooler = MeanPooling()

        ##self.liner -> sequential
        self.phising_classifier = nn.Sequential(
            nn.Dropout(p=0.3), #Dropout Layer추가 >> 일반화 성능
            nn.Linear(in_features=self.out, out_features=2),
        )

    def forward(self, input_ids, attention_mask, labels=None, token_type_ids=None):
        x = self.base_model(input_ids=input_ids, attention_mask=attention_mask)[0]

        #pooling적용
        pooling_x = self.mean_pooler(x, attention_mask=attention_mask)[0]

        #pooling적용한 결과와 base_model을 적용한 결과 값을 합침
        # pooling을 적용함으로써 누락된 정보가 발생하는데, 이를 최소화 하기위해서 기존의 데이터와 더함
        x=x + pooling_x

        # 위에서 합친 값을 기반으로 phising 여부 분류
        phising_output = self.phising_classifier(x[:,0,:].view(-1,self.out))
        return phising_output

In [18]:
# Custom Model
Custom_Model=CustomModel()
print(Custom_Model)

CustomModel(
  (base_model): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), 

In [19]:
# Original Model
Origin_Model=AutoModel.from_pretrained('monologg/kobigbird-bert-base', config=config)
print(Origin_Model)

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0-11): 12 x BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=

## Trainer arg

In [20]:
# Trainer arguments
lr = 2e-5
stop = 3
epoch = 10 #1000
batch = 16
seed = 42

## Focal Loss

In [21]:
class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

def compute_metrics(pred):
    # label = [[cls1,cls2,...],]
    # preds = n list
    focal_loss = FocalLoss()
    labels = pred.label_ids
    preds = pred.predictions
    f1 = []
    focal = []

    #labels=labels[labels==1]
    #preds=preds[preds==1]

    f1.append(f1_score(y_true = labels, y_pred = preds, average='weighted'))
    return {
        #'focal': sum(focal),
        'f1-sum': sum(f1)
        #'TP-Ratio':((labels==preds).sum())/len(preds)
        #'TP-Phising':(((labels==1) and (preds==1)).sum())/len((preds==1).sum())
    }

## Custom Trainer

In [22]:
# Define trainer
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        # forward pass
        labels = inputs.pop("labels").to(torch.int64)

        is_phising_logit = model(**inputs)

        # focal loss
        criterion = {
            'is_phising' : FocalLoss().to(device)
        }
        # labels = labels.type(torch.float).clone().detach()
        loss = criterion['is_phising'](is_phising_logit, labels)

        outputs = None, torch.argmax(is_phising_logit, dim = 1)
        return (loss, outputs) if return_outputs else loss

# Fold Training

In [23]:
##메모리 정리
import gc
torch.cuda.empty_cache()

gc.collect()

0

In [24]:
label = LabelEncoder()
label.fit(train['label'])

def encoding(X_train, X_val):

    X_train['label'] = label.transform(X_train['label'])
    X_val['label'] = label.transform(X_val['label'])

    train_labels = {
        'is_phising' : X_train['label'].values
    }

    val_labels = {
        'is_phising' : X_val['label'].values
    }
    return train_labels, val_labels

In [ ]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'ej.park'
print(f'hidden_layers : {config.num_hidden_layers}')
config.num_hidden_layers = 12
print(f'now_hidden_layers : {config.num_hidden_layers}')


kf = KFold(n_splits=5, random_state=seed, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    print(f'Round {i}')
    X_train, X_val = train.loc[train_index, :], train.loc[test_index, :]
    train_labels, val_labels = encoding(X_train, X_val)
    token_train, token_val = tokenizer(X_train.comment.tolist(), padding=True, truncation=True, max_length=length), tokenizer(X_val.comment.tolist(), padding=True, truncation=True, max_length=length)
    train_dataset, val_dataset = CustomDataset(token_train, train_labels), CustomDataset(token_val, val_labels)
    model = CustomModel()
    model.to(device)
    args = TrainingArguments(run_name = f'fold_{i}',                                # 모델이름
                             output_dir= f"fold_{i}",                               # 모델저장경로
                             evaluation_strategy="steps",                           # 모델의 평가를 언제 진행할지
                             eval_steps=100,                                         # 100 스텝 마다 모델 평가
                             save_steps=100,                                         # 100 스텝 마다 모델 저장
                             save_total_limit = 2,                                  # 저장할 모델의 갯수
                             logging_steps=100,                                      # 학습로스 로깅
                             per_device_train_batch_size=batch,                     # GPU에 학습데이터를 몇개씩 올려서 학습할지
                             per_device_eval_batch_size=batch,                      # GPU에 학습데이터를 몇개씩 올려서 평가할지
                             gradient_accumulation_steps=16,                        # 가상배치
                             num_train_epochs=epoch,                                # 전체 학습 진행 횟수
                             learning_rate=lr,                                      # 학습률 정의
                             seed=seed,                                             # seed
                             load_best_model_at_end=True,                           # 평가기준 스코어가 좋은 모델만 저장할지 여부
                             #fp16=True,
                             do_train=True,
                             do_eval=True,
                             # metric_for_best_model
                             # greater_is_better = True,
    )
    trainer = CustomTrainer(model=model,
                            args=args,                                                        # args
                            train_dataset=train_dataset,                                      # 학습데이터
                            eval_dataset=val_dataset,                                         # validation 데이터
                            compute_metrics=compute_metrics,                                  # 모델 평가 방식
                            callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],) # callback
    trainer.train()
    del model
    del trainer
    gc.collect() # python 자원 관리
    torch.cuda.empty_cache() # gpu 자원관리

hidden_layers : 12
now_hidden_layers : 12
Round 0


Step,Training Loss,Validation Loss,F1-sum
100,0.048800,0.000506,1.000000
200,0.001200,0.000769,0.999850
300,0.000800,0.001530,0.999850
400,0.000800,0.005215,0.998346


Round 1


Step,Training Loss,Validation Loss,F1-sum
100,0.046600,0.006583,0.998195
200,0.001200,0.001042,0.999699
300,0.000500,0.001022,0.999699
400,0.000300,0.001043,0.999699
500,0.000300,0.001205,0.999549
600,0.000200,0.001187,0.999699


Round 2


...

Round 3


Step,Training Loss,Validation Loss,F1-sum
100,0.051400,0.000587,1.000000
200,0.001000,0.001342,0.999399
300,0.000900,0.007268,0.998647
400,0.000500,0.000221,1.000000
500,0.000300,0.005793,0.999098
600,0.000300,0.003035,0.999549
700,0.000300,0.003173,0.999549


Round 4


Step,Training Loss,Validation Loss,F1-sum
100,0.047600,0.001912,0.999549
200,0.001100,0.001946,0.999549


RuntimeError: ignored

# Predict

In [25]:
def recent_file(path):
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌.
    for f_name in os.listdir(f"{path}"):
        written_time = os.path.getctime(f"{path}/{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고,
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    return f"{path}/{recent_file_name}"

In [26]:
gc.collect() # python 자원 관리
torch.cuda.empty_cache() # gpu 자원관리

In [27]:
gc.collect() # python 자원 관리
torch.cuda.empty_cache() # gpu 자원관리
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenized = tokenizer(test['comment'].tolist(), padding=True, truncation=True, max_length=length, return_tensors="pt")
test_dataset = CustomDataset(tokenized, None)
test_args = TrainingArguments(
    output_dir = './',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 112,
    dataloader_drop_last = False
)

tmp = 0
while os.path.isdir(f'fold_{tmp}'):
    tmp += 1

test_results = []
for i in range(tmp):
    print(f'Round {i}')
    # model = AutoModel.from_pretrained(recent_file('custom_model'), config=config)
    model = CustomModel().to(device)
    model.load_state_dict(torch.load(f"{recent_file(f'fold_{i}')}/pytorch_model.bin"))
    trainer = CustomTrainer(
                  model = model,
                  args = test_args,
                  compute_metrics = compute_metrics)
    test_results.append(trainer.predict(test_dataset))
    gc.collect() # python 자원 관리
    torch.cuda.empty_cache() # gpu 자원관리
    del model
    del trainer

Round 0


<ipython-input-14-a1961c41a758>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Round 1


Round 2


Round 3


FileNotFoundError: ignored

In [28]:
test['preds'] = list(sum(list(map(lambda x: x.predictions, test_results))))
test['preds'] = list(map(lambda x : label.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions, test_results)))))

## 피싱여부
test['preds'] = list(map(lambda x : x[0], test['preds']))
test.head()

,comment,label,preds
0,에콰도르 교도소에서 폭동 일어나서 여명 피살됐대요 아이고 왜 그러노 왜 폭...,0,0
1,네 그런데 제가 전화해보니까 없으시다고 하던데요 어느 부분에 연락을 주신 건가요 저...,1,1
2,지금 다 읽었습니다 내용 중에 이해 안 가시거나 의문의 부분이 있으십니까 그런 건 ...,1,1
3,중국은행 업무문의합니다 중국은행 공인인증서 기간연장 어떻게 하죠 코로나로인해 ...,0,0
4,네 네 김진화 드립니다 통화하셨어요 그쪽혹시 뭐 다른 거 물어보지는 않던가요 말씀 ...,1,1


In [29]:
print('f1 score(weighted) : ',f1_score(test['label'],test['preds'],average='weighted'))
print('f1 score(macro) : ',f1_score(test['label'],test['preds']))
print('True-Positive : ', (test['label']==test['preds']).sum() / len(test['label']))

f1 score(weighted) :  0.9998798002934061
f1 score(macro) :  0.9997923156801661
True-Positive :  0.9998798076923077


In [30]:
#실제로 피싱인 데이터인데, 정상으로 잘못 예측한 비율
test_phising=test[test['label']==1]
print('FP-Phising : ', (test_phising['preds']==0).sum() / len(test_phising['label']))

FP-Phising :  0.0004152823920265781


In [31]:
#실제로 피싱인 데이터인데, 피싱으로 올바르게 예측한 비율
test_phising=test[test['label']==1]
print('TP-Phising : ', (test_phising['preds']==1).sum() / len(test_phising['label']))

TP-Phising :  0.9995847176079734


In [32]:
test_phising[test_phising['preds']==0]

,comment,label,preds
6469,아이디가 특정되었습니다 당신은 회사에서 일하고 있습니까 아니면 개인사업주 명의의 사...,1,0


In [33]:
#실제로 정상인 데이터에서 정상으로 올바르게 예측한 비율
test_normal=test[test['label']==0]
print('TN-Normal', (test_normal['preds']==0).sum() / len(test_normal['label']))

TN-Normal 1.0


In [34]:
#실제로 정상인 데이터에서 피싱으로 예측한 비율
test_normal=test[test['label']==0]
print('FN-Normal', (test_normal['preds']==1).sum() / len(test_normal['label']))

FN-Normal 0.0


### Inference

In [68]:
def inference(text):
  text_list=['Nothing']
  text_list.append(text)
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  tokenized = tokenizer(text_list, padding=True, truncation=True, max_length=length, return_tensors="pt")
  test_dataset = CustomDataset(tokenized, None)
  test_args = TrainingArguments(
      output_dir = './',
      do_train = False,
      do_predict = True,
      per_device_eval_batch_size = 112,
      dataloader_drop_last = False
  )

  tmp = 0
  while os.path.isdir(f'fold_{tmp}'):
      tmp += 1

  test_results = []
  for i in [0,1,2]:#range(tmp):
    print(f'Round {i}')
    # model = AutoModel.from_pretrained(recent_file('custom_model'), config=config)
    model = CustomModel().to(device)
    model.load_state_dict(torch.load(f"{recent_file(f'fold_{i}')}/pytorch_model.bin"))
    trainer = CustomTrainer(
                    model = model,
                    args = test_args,
                    compute_metrics = compute_metrics)
    test_results.append(trainer.predict(test_dataset))
    gc.collect() # python 자원 관리
    torch.cuda.empty_cache() # gpu 자원관리
    del model
    del trainer

  result = list(sum(list(map(lambda x: x.predictions, test_results))))
  result = list(map(lambda x : label.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions, test_results)))))

  ## 피싱여부
  result = list(map(lambda x : x[0], result))

  return result[-1]

In [70]:
sample='''


'''

inference(sample)

Round 0


<ipython-input-14-a1961c41a758>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Round 1


Round 2


0